In [1]:
import whisperx
import torchaudio
device = "cuda" 
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

/home/kenfus/miniconda3/envs/whisperx/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kenfus/miniconda3/envs/whisperx/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
torchvision is not available - cannot save figures


In [2]:
MP3_PATH = 'Schönleber_1.mp3'
MODEL_PATH = "Systran/faster-whisper-large-v2"

In [3]:
vad_options = {'vad_onset': 0.5, 'vad_offset': 0.3}
#asr_options = {'temperatures': 0}

In [4]:
# 1. Transcribe with original whisper (batched)
model = whisperx.load_model(MODEL_PATH, device, compute_type=compute_type, language='de')#,  vad_options=vad_options) #asr_options=asr_options)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.2.2+cu121. Bad things might happen unless you revert torch to 1.x.


In [5]:
import logging
# Configure logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Create a file handler that logs even debug messages
file_handler = logging.FileHandler('application.log')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))

# Add the file handler to the logger
model.model.logger.addHandler(file_handler)

In [6]:
audio = torchaudio.load(MP3_PATH)

In [7]:
vad = model.vad_model({"waveform": audio[0], "sample_rate": audio[1]})

In [9]:
audio = whisperx.load_audio(MP3_PATH)
result = model.transcribe(audio, batch_size=batch_size, language="de", print_progress=True, combined_progress=True)
print(result["segments"]) # before alignment

Progress: 1.61%...
Progress: 3.23%...
Progress: 4.84%...
Progress: 6.45%...
Progress: 8.06%...
Progress: 9.68%...
Progress: 11.29%...
Progress: 12.90%...
Progress: 14.52%...
Progress: 16.13%...
Progress: 17.74%...
Progress: 19.35%...
Progress: 20.97%...
Progress: 22.58%...
Progress: 24.19%...
Progress: 25.81%...
Progress: 27.42%...
Progress: 29.03%...
Progress: 30.65%...
Progress: 32.26%...
Progress: 33.87%...
Progress: 35.48%...
Progress: 37.10%...
Progress: 38.71%...
Progress: 40.32%...
Progress: 41.94%...
Progress: 43.55%...
Progress: 45.16%...
Progress: 46.77%...
Progress: 48.39%...
Progress: 50.00%...
[{'text': ' Ich rufe an, weil ich nicht mehr zum Arzt komme.', 'start': 1.459, 'end': 19.155}, {'text': ' Jetzt habe ich das Problem von mir gar nicht geschoben. Seit acht Wochen habe ich so einen Husten, der geht und geht und geht einfach nicht weg. Und zwar habe ich das Gefühl, normalerweise wenn ich huste, oder wenn ich das früher hatte, dann merke ich, wie irgendwann mal das Zeug

In [10]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment


[{'start': 1.479, 'end': 3.181, 'text': ' Ich rufe an, weil ich nicht mehr zum Arzt komme.', 'words': [{'word': 'Ich', 'start': 1.479, 'end': 1.579, 'score': 0.209}, {'word': 'rufe', 'start': 1.599, 'end': 1.759, 'score': 0.378}, {'word': 'an,', 'start': 1.779, 'end': 1.859, 'score': 0.284}, {'word': 'weil', 'start': 1.879, 'end': 2.32, 'score': 0.441}, {'word': 'ich', 'start': 2.42, 'end': 2.5, 'score': 0.172}, {'word': 'nicht', 'start': 2.52, 'end': 2.7, 'score': 0.307}, {'word': 'mehr', 'start': 2.72, 'end': 2.84, 'score': 0.355}, {'word': 'zum', 'start': 2.86, 'end': 2.92, 'score': 0.101}, {'word': 'Arzt', 'start': 2.96, 'end': 3.04, 'score': 0.02}, {'word': 'komme.', 'start': 3.06, 'end': 3.181, 'score': 0.46}]}, {'start': 20.128, 'end': 23.57, 'text': ' Jetzt habe ich das Problem von mir gar nicht geschoben.', 'words': [{'word': 'Jetzt', 'start': 20.128, 'end': 20.388, 'score': 0.193}, {'word': 'habe', 'start': 20.408, 'end': 20.548, 'score': 0.432}, {'word': 'ich', 'start': 20.5

In [11]:
import pysubs2

subs = pysubs2.load_from_whisper(result)
subs.save(MP3_PATH.replace('.mp3', '.srt'))